In [ ]:
!nvidia-smi

In [ ]:
!pip install --upgrade pip > /dev/null
!pip install transformers > /dev/null
!pip3 install sentencepiece > /dev/null

In [ ]:
import pandas as pd
import numpy as np
import torch
import transformers

from transformers import BertTokenizer
from tqdm import tqdm
tqdm.pandas()

In [ ]:
#DATADIR = "drive/MyDrive/atma10/input/"
#OUTPUTDIR = "drive/MyDrive/atma10/feature/"
DATADIR = "../input/"
OUTPUTDIR = "../feature/"

In [ ]:
train_data = pd.read_csv(DATADIR + "train.csv")
test_data = pd.read_csv(DATADIR + "test.csv")

all_df = pd.concat([train_data, test_data], axis=0).reset_index(drop=True)

In [ ]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
from transformers import AutoModelForSequenceClassification
from transformers import TFAutoModelForSequenceClassification
from scipy.special import softmax
import csv
import urllib.request

In [ ]:
task='sentiment'

class BertSentiment:
    def __init__(self):
        self.device = 'cuda' if torch.cuda.is_available() else 'cpu'
        self.model_name = f"cardiffnlp/twitter-roberta-base-{task}"
        self.tokenizer = AutoTokenizer.from_pretrained(self.model_name)
        self.bert_model = AutoModelForSequenceClassification.from_pretrained(self.model_name)
        self.bert_model = self.bert_model.to(self.device)
        self.max_len = 128
        print(self.device)


    def predict(self, sentence : str) -> np.array:
        if sentence is np.nan:
            return np.nan

        encoded_input = self.tokenizer(sentence, return_tensors='pt').to(self.device)
        output = self.bert_model(**encoded_input)
        scores = output[0][0].detach().cpu().numpy()

        return scores

In [ ]:
import re
def remove_pad_token(s):
    if s is np.nan:
        return np.nan
    
    clean_s = re.sub("<pad>", "", s)
    if clean_s[0] == " ":
        clean_s = clean_s[1:]
    return clean_s

In [ ]:
# 1columnあたり5~10分

BS = BertSentiment()
for c in ["transed_description", "transed_long_title"]:
    _df = pd.read_csv(f"drive/MyDrive/atma10/feature/{c}.csv")
    try:
        _df = _df.drop("Unnamed: 0", axis=1)
    except:
        pass
    print(_df.shape)

    _df[c] = _df[c].apply(lambda x:remove_pad_token(x))
    predict = _df[c].progress_apply(lambda x: BS.predict(x))
    output = pd.concat([_df["object_id"],
                        pd.DataFrame(predict.values.tolist(), columns=["label1", "label2", "label3"])], axis=1)
    output.to_csv(OUTPUTDIR + f"{c}_tweet_{task}.csv", index=False)


for c in ["long_title", "title", "principal_maker"]:
    predict = all_df[c].progress_apply(lambda x: BS.predict(x))
    output = pd.concat([all_df["object_id"],
                        pd.DataFrame(predict.values.tolist(), columns=["label1", "label2", "label3"])], axis=1)
    output.to_csv(OUTPUTDIR + f"{c}_tweet_{task}.csv", index=False)